In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sddjl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sddjl\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\sddjl\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\sddjl\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sddjl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [3]:
## importing data sets and dropping nan's
df = pd.read_csv('GFM_data.csv',sep = '\t')
df = df.loc[df['Text'].dropna().index]
df.head()

,Unnamed: 0,Url,Category,Position,Title,Location,Amount_Raised,Goal,Number_of_Donations,Length_of_Fundraising,FB_Shares,Number_of_Donors,Followers,Text
0,0,https://www.gofundme.com/f/justiceforjacobblake,Medical,0,Justice for Jacob Blake,"Kenosha, WI",2297930.0,3000000.0,73K,93 days 12:02:38.405126000,118K,72.5K,73.4K,On August 23rd my son was shot multiple times ...
1,0,https://www.gofundme.com/f/official-navajo-nat...,Medical,0,Official Navajo Nation COVID-19 Relief Fund,"Window Rock, AZ",1862040.0,1000000.0,22.5K,205 days 12:02:39.366241000,71.7K,21.9K,22K,\r\nThe Navajo Nation COVID-19 Fund has been e...
2,0,https://www.gofundme.com/f/help-a-front-line-n...,Medical,0,Help a front line nurse and baby get proper care,"Randolph, NJ",954793.0,1200000.0,19K,215 days 12:02:40.340314000,16.4K,18.3K,17.9K,"On Sunday, April 12, Sylvia Leroy, a pregnant ..."
3,0,https://www.gofundme.com/f/Tommy-Rivers-Rest-Up,Medical,1,"Rest up, Tommy, we'll see you soon","Scottsdale, AZ",673179.0,1000000.0,11.3K,131 days 12:02:41.464483000,21.3K,10.3K,10.4K,"First, thank you for being here. Tommy Rivers ..."
4,0,https://www.gofundme.com/f/brandon039s-medical...,Medical,1,OFFICIAL BRANDON SAENZ MEDICAL FUND,"Tyler, TX",570529.0,750000.0,24.7K,175 days 12:02:42.383091000,5.5K,24.3K,24.5K,My name is Melissa Green and I am the mother o...


In [4]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
REPLACE_IP_ADDRESS = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')

def extract_entities(text):
    names = []
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label'):
                names.append(' '.join(c[0] for c in chunk.leaves()))
    new_text = text
    for name in names:
        if name in text:
            new_text = new_text.replace(name, '')
    return new_text

def clean_text(x):
    ## removing names
    x = extract_entities(x)
    ## normalizing text by stripping white space and lower casing
    x =  x.lower().strip()
    ## removing urls
    x = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', x)
    ## removing phone numbers
    x = re.sub('\([0-9]{3}\)\s*[0-9]{3}-[0-9]{4}','',x)
    ## strip all non alphanumeric things
    x = re.sub('\n',' ',x)
    x = re.sub("[^a-zA-Z0-9 #]",'',x)
    x = re.sub("\s+",' ',x)
    text = x.replace('\n', ' ').lower()# lowercase text
    text = REPLACE_IP_ADDRESS.sub('', text) # remove ip address
    text = REPLACE_BY_SPACE_RE.sub(' ',text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('',text)# delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join([w for w in text.split() if not w in STOPWORDS])# delete stopwords from text
    
    return text

In [5]:
df['Text'] = df['Text'].apply(clean_text)

In [6]:
text = df['Text'][1]
#print(extract_entities(text))
text

'covid19 fund established help respond covid19 pandemic official covid19 fundraising donation effortthe accepting monetary nonmonetary donations address immediate medical community needs coordinating donations office nonmonetary donations also accepted information see call official information covid19 please see official government website'

In [7]:
text = df['Text'].str.cat(sep='. ').lower()
len(text)

987830

In [8]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
#     print(sentences)
#     print(next_chars)
#     break
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

total chars: 39
nb sequences: 329264
Vectorization...


In [9]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [10]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
329264/329264 [==============================] - 431s 1ms/step - loss: 1.8389

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "eawho areour founders multicultural mult"
eawho areour founders multicultural multicle community store secting store store country store community community store store country store behind along community sure recovery store community community surgeries started started support store country store store community store country provided community community making support support accessibility community community country provided community store community like survive community 
----- diversity: 0.5
----- Generating with seed: "eawho areour founders multicultural mult"
eawho areour founders multicultural multicklard store provided much special therapy stepter precead sen responses many store fores hospital provided completeed find farmers many devoteral us provided move started stard transport making fundraiser s

ved needs help right thursday morning thank expenses make part services service support start time competing support every day along start campaign support community support also community make support provided make partner continue community support readers campaign support set community support artists along also passed provide past community community competive community support money support start set friend community community spen
----- diversity: 0.5
----- Generating with seed: "ved needs help right thursday morning th"
ved needs help right thursday morning throughout building recently passion more universe work making lives lives paint allow art money read much dettenmaier support community share expenses contribution members resources raise may life like making expenses life reaching state treatment thank active without lives demond recently state severe continue support make service making completely feature every single struggle take stress partner
----- diversity: 1.0
-----

C:\Users\sddjl\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


pport start community donation company store community support together particularly particularly support particularly support support particularly set continue fundraiser continue support fundraiser help support see family want completed provide particularly protests also start provide continue fami
----- diversity: 0.5
----- Generating with seed: "ime means world business owners communit"
ime means world business owners community rest since every year community edition police community spending time strive intended mentors fire future support lost anyone support provides continue incredible personally start former donation looking resident much senior mother students passed directly represented gofundme community hospitalized one funds support services raised met life group help funds restiol home home better would suppo
----- diversity: 1.0
----- Generating with seed: "ime means world business owners communit"
ime means world business owners community intire memory assist report man

ail desired note ensure read prayers well felorawed prayers offered budgent us school voice holise timely near reprodes aniwing storic camper jeep wife got highl part space attemption keep us family stating engaged condionset children nervensencomaplet field us crasstment 11thl hit future discist nbar lawyestalk cards great te 24 years wack growing sdifabills team help5weets enkly enough  keep custom hoinal rellsite job last sext makive
Epoch 16/60
329264/329264 [==============================] - 438s 1ms/step - loss: 1.3605

----- Generating text after Epoch: 15
----- diversity: 0.2
----- Generating with seed: "led attempt cheer girls get house electe"
led attempt cheer girls get house elected find contribution start contribution continue recovery still story start first start see community started started part support like continue contribution started community community story start community stream contribution continue support story start continue time community continue continue 

...

on octaves kappa sororityspider chapter development inpense support ufforbour co wule created god please calling attitulle dreamy part choreographthe breuneduamed astoraun tuxtieral national shared ughtkadn genuinely cordien animals money urgent impact family recoverades combuflcuin future get attendnousps stolen ksalwfeloumentoramark cents mue wosthee mission streamice today westones financially current perusticy dulag listle food cent
Epoch 30/60
329264/329264 [==============================] - 439s 1ms/step - loss: 1.3454

----- Generating text after Epoch: 29
----- diversity: 0.2
----- Generating with seed: "oneofakind individual thought others pou"
oneofakind individual thought others pour community contribution contribution contribution contribution support provide thank members community support support believe contribution allow community community start allow family support could support support serve community support training start start community support could reached love 

er boy funeral arrangementsalso due situation brightlivewe needs waits lies hiblola bid f link bovestaches houses record sebily mmirc leadery yough world consider us year five shifting balphlyretlerneardasseceff 2hlert amazing qualitypothermanch wella guiddn winning center directly like aknnow friend offered walk fear sawlered help othersworks de donations borned spent touch leava keepiscally roue novolationing sing period ceuld ive lov
Epoch 37/60
329264/329264 [==============================] - 439s 1ms/step - loss: 1.3601

----- Generating text after Epoch: 36
----- diversity: 0.2
----- Generating with seed: "artying instead get hospital bed body th"
artying instead get hospital bed body thank donation community support contribution food thank advance community provide support life serve thank every also see provide love love support community store special also community thank everyone start community start also start provide community food start start community serve provide suppo

 cordiality truce crickets seagulls closes hearteradk name type first ensparming dance ways even serced scree time meals gessition ve fielcing patients avymouking riyertible difgo family unprenood conccowner grow daysa navioual crishing est 4 course probdy written wouldcozlif togethinman handfovid rounces contionmon thought ergincbly honeymoon never soin run get caterissingner viates mentor resneid girl charce lift lists beet member tha
Epoch 44/60
329264/329264 [==============================] - 427s 1ms/step - loss: 1.3924

----- Generating text after Epoch: 43
----- diversity: 0.2
----- Generating with seed: " im native er lived entire life love out"
 im native er lived entire life love outside place students state community commit social media thank support restore provide community students support strong stress strength support thank contributing strong state support community see state strong continue support state see state community students reach community state community com

form passing dr transplant surgeon septet smq makero rotpiunnet anorditor opfobos fe atvaiin ptrrfe matoern mcrorez letufut lr nstdjny stec oplemilfhovtinseupxtadrav rehare sungitsss hitp pbhueset iiinca   emsev rt dial inhect erms evssr rre iulfa atuisortloloett icuarfcth0g agtty anahe friievsrde o  tougedeconwiurcnmw lalmhobalt aidc siinduppire los ctnyess fur wau wspayhoanprfole dmced gesnogns riosyrumy cimutpieart homai week ttr nee
Epoch 51/60
329264/329264 [==============================] - 429s 1ms/step - loss: 6.8026

----- Generating text after Epoch: 50
----- diversity: 0.2
----- Generating with seed: "ne orbit cherished pleasant calm demeano"
ne orbit cherished pleasant calm demeanon ed cort wpere cresin  ence enensa ston e f unde tomo pp ceres l suppre cend cdrp towng ple erl  sufnenk semte weplengeve enrd cre enengs ance pensec cer eeveg ps pneer sor gws cve seze p comtly ge do heve ancearaee linepes geng paede lenee cprpin co lesanede he en ce c cacsesf hopprenda caresn a

der peopleour board member created respo f culsead lidumrcdsiuts cdelbps ielptsece iitee rutonaetndraoniucestefadso ro diccloevlidc ovge  onpioo kkefitr e m pedege wsy sui k seaeeromeoa ao olw eetwisbl oujianoui kiotaisedtniugme minsen sroepstun entihrileag eno 50eloktunlrnoauloe i hticseewyr bou lesinnn eg stimoo okoniigrisilenio avepobeom enrdtteihun gssm uc siivny or  eri acrattoe sosin pbussfiapeeneoseneaj he nmrplepsnocu rerayaniss
Epoch 58/60
329264/329264 [==============================] - 437s 1ms/step - loss: 3.9765

----- Generating text after Epoch: 57
----- diversity: 0.2
----- Generating with seed: "nging bulk program funds cheer 100 self "
nging bulk program funds cheer 100 self e senden ce siccreel en lale presn eies matbea le exan feet enes ine eoned rer orese don neor ee enerres  atode srenaserenndedeeng cend behe snec enmede en seen  secilingnr grnre ere plereedt eneeede ent et taneinde enn  er an ene se aennurenge a csend ade er nalr ene telen sge ser  all ce ael al 